# Wstęp do sztucznych sieci neuronowych
### Czym są sieci neuronowe?
Najprościej -- uniwersalnym aproksymatorem funkcji. To, co jest ważne, to że będziemy uczyć sieć neuronową aproksymować pewną (wybraną przez nas) funkcję.


### A w praktyce?
W praktyce sieci neuronowe prezentowane są jako złożenie kombinacji liniowych i nieliniowych przekształceń. Brzmi skomplikowanie, dlatego zacznijmy bardzo prostego przykładu, sieci z jednym neuronem:


![](https://www.researchgate.net/publication/286020106/figure/fig13/AS:328309167673363@1455286414002/Basic-artificial-neural-network-cell-artificial-neuron.png)
Źródło: Hüseyin Ceylan, An Artificial Neural Networks Approach to Estimate
Occupational Accident: A National Perspective for Turkey, 2014

W prakyce sieci neuronowe wizualizowane są przez tego typu obrazki:
![](https://visualstudiomagazine.com/articles/2014/11/01/~/media/ECG/visualstudiomagazine/Images/2014/11/1114vsm_mccaffreyfig2.ashx)
Źródło: https://visualstudiomagazine.com/articles/2014/11/01/use-python-with-your-neural-networks.aspx

Spójrzmy teraz na sieć w nieco inny sposób niż na powyższych obrazkach. Pomyślmy, jak opisać ją wzorem. Z pominięciem składowej stałej możemy powyższą sieć opisać wzorem:

$$ y = W_2 \cdot f(W_1 \cdot x) $$

Gdzie:  
$ y $  -- wektor wyjść  
$ x $  -- wektor wejść  
$ f() $ -- nieliniowa funkcja $ \mathbb{R}^n \to \mathbb{R}^n $  
$ W_1 $, $ W_2 $ -- macierze wag odpowiednich warstw

Dlaczego ważny jest zapis macierzowy? Komputer bardzo dobrze zrównolegla obliczenia macierzowe. W szczególności karty graficzne specjalizują się w obliczeniach macierzowych. Dlatego trening sieci na karcie graficznej jest zazwyczaj kilkadziesiąt razy szybszy niż na procesorze.

### Jak wytrenować sieć neuronową?
Do tej pory mówiliśmy o tym, jak sieci neuronowe wyznaczają wyjście dla zadanego wejścia. Ale jak je nauczyć? Odpowiedzią jest algorytm wstecznej propagacji gradientu. Na wysokim poziomie działa to tak:
* Dajemy sieci wejście i mówimy: zgadnij, jaka jest odpowiedź
![](https://hmkcode.com/images/ai/bp_forward.png)
* Sieć zwraca nam wyjście, a my liczymy ile się pomyliła
![](https://hmkcode.com/images/ai/bp_error.png)
* Wyznaczamy różniczki błędu po poszczególnych wagach. Liczymy to w dość sprytny sposób, ale nie będziemy się w to teraz wgłębiać
* Znając tę różniczki, możemy powiedzieć jak mocno poszczególne wagi wpłynęły na uzyskany błąd. Dlatego aktualizujemy wagi według wzoru:
$$ W_{k, i, j} = W_{k, i, j} - a \cdot {{\partial E}\over{\partial W_{k, i, j}}}$$
Gdzie:  
$W_{k, i, j}$ -- waga połączenia z $i$-tego neuronu $k-1$ warstwy do $j$-tego neuronu $k$-tej warstwy  
$E$ -- obliczona "różnica" między wartością zwróconą a tą, której się spodziewaliśmy  
$a$ -- stała uczenia

W rzeczywistości takie uczenie byłoby bardzo niestabilne, ponieważ każdy punkt ze zbioru uczącego "ciągnąłby" parametry w swoją stronę. Dlatego uczymy w tak zwanych mini-batchach składających się z kilkunastu -- kilkuset punktów jednocześnie. Pochodne obliczone dla każdego z punktów z batcha dodajemy do siebie i otrzymujemy "średni kierunek poprawiający". W tej sposób sieć uczy się znacznie bardziej stabilnie.

### A jak to zaprogramować?
* Można napisać wszystko ręcznie -- zapewne czeka Was jeszcze taki projekt.
* Można skorzystać z dedykowanych bibliotek, na przykład `pytorch`, `tensorflow` czy `keras` 
* Można skorzystać z `sklearn.neural_network`

### Przykład 1. -- klasyfikacja
W związku z tym, że sieć neuronowa może mieć wiele wyjść, bardzo naturalne jest jej wykorzystanie do zadania klasyfikacji wieloklasowej. Pomysł jest taki, żeby $i$-te wyjście mówiło o prawdopodobieństwie przynależności do $i$-tej klasy. Aby to osiągnąć, normalizuje się wyjścia funkcją sofmax:
$$ {{e^{y_i}}\over {\sum_{j=1}^K e^{y_j}}} $$

Gdzie:  
$y_i$ -- $i$-te wyjście  
$K$ -- liczba klas

Zbiór mnist zawiera ręcznie pisane cyfry.
![](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)
Żródło: https://en.wikipedia.org/wiki/MNIST_database

In [ ]:
import os
import numpy as np
import pandas as pd
import time

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from sklearn.metrics import confusion_matrix

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()
plt.imshow(X_train_full[0], cmap='binary')
y_train_full.shape

In [ ]:
X_train, y_train = X_train_full[:50_000] / 255., y_train_full[:50_000]
X_valid, y_valid = X_train_full[50_000:] / 255., y_train_full[50_000:]

Do stworzenia modelu wykorzystamy model sekwencyjny. W ten sposób dane będą przetwarzane po kolei przez wszystkie warstwy, które dodamy. 

Poza interfejsem sekwencyjnym w bibliotece Tensorflow występuje także interfejs funkcjonalny oraz podklasowy, ale o nich dzisiaj nie będzie.




In [ ]:
def build_model(hidden_layer_size):
    model = Sequential()
    model.add(Flatten(input_shape=(28,28)))
    model.add(Dense(hidden_layer_size, activation='relu'))
    model.add(Dense(hidden_layer_size, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

Tensorflow wymaga kompilacji modelu. Przy okazji musimy określić funkcję błędu, optymalizator oraz metryki służące do oceny modelu. W naszym przypadku wykorzystamy optymalizator Stochastic Gradient Descent (SGD). Musimy także podać krok uczenia - learninig rate. **Jest to najważniejszy hiperparametr przy uczeniu modeli głębokich.** Najwięcej czasu powinno być poświęcone właśnie na jego poprawne dobranie. W ogólności nie musi on być stały, ale zmieniać się np wraz z numerem iteracji fazy uczenia.

In [ ]:
model = build_model(256)
lr = 2e-1
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=SGD(learning_rate=lr),
             metrics=['accuracy'])
model.summary()

In [ ]:
h = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=64)

In [ ]:
plt.plot(h.history['loss'], label='loss')
plt.plot(h.history['val_loss'], label='val_loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.plot(h.history['accuracy'], label='accuracy')
plt.plot(h.history['val_accuracy'], label='val_accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)

conf_m = confusion_matrix(y_test, y_pred)

row_sums = conf_m.sum(axis=1)
conf_m = conf_m / row_sums[:, np.newaxis]

mask = np.ones(conf_m.shape, dtype=bool)
np.fill_diagonal(mask, 0)
max_value = conf_m[mask].max()

plt.figure(figsize = (15,12))
sns.heatmap(conf_m, annot=True, vmax=max_value, cmap="YlGnBu")

## Przykład 2

In [ ]:

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
tensorboard_path = os.path.join(os.curdir, 'my_logs')
def get_run_logdir():
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(tensorboard_path, run_id)

In [ ]:
model = Sequential([
    Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    Dense(1)
])
model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))

tensorboard_cb = tf.keras.callbacks.TensorBoard(get_run_logdir())
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

h = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), 
          callbacks=[tensorboard_cb, earlystop_cb])


In [ ]:
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

### Kiedy używać sieci neuronowych:

1. Kiedy mamy duży zbiór danych
2. W zadaniach podobnych do tych, w których ktoś już z powodzeniem użył sieci neuronowych
3. Jeśli istnieją architektury dostosowane do naszego problemu
4. Kiedy nie zależy nam na interpretowalności
5. Gdy mamy dostępną dużą moc obliczeniową do treningu
6. Kiedy mamy dużo czasu na przygotowanie modelu
7. Gdy rozwiązanie na produkcji nie musi działać bardzo szybko lub jesteśmy w stanie zapewnić dużą moc obliczeniowa również na produkcję
8. Gdy zależy nam na (pewnej) odporności na szum na wejściu
9. Jeśli chcemy "wyciągnąć" reprezentację z warstwy ukrytej
10. Jeśli zależy nam na pewnej elastyczności wynikającej z modularnej budowy

## Ćwiczenie 
Klasyfikacja obrazów - Cifar-10

In [ ]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
plt.imshow(X_train[0], cmap='binary')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=5_000)
for s in [X_train, y_train, X_val, y_val, X_test, y_test]:
    print(s.shape)